In [1]:
import pandas as pd
import numpy as np

### Loading and Cleaning the Data

In [5]:
df = pd.read_csv('/content/data_new.csv')
data_df = df.reset_index()
data_df.info()
data_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48784 entries, 0 to 48783
Columns: 161 entries, index to gagne_sum_t
dtypes: float64(22), int64(138), object(1)
memory usage: 59.9+ MB


,index,risk_score_t,program_enrolled_t,cost_t,cost_avoidable_t,bps_mean_t,ghba1c_mean_t,hct_mean_t,cre_mean_t,ldl_mean_t,...,trig_min-high_tm1,trig_min-normal_tm1,trig_mean-low_tm1,trig_mean-high_tm1,trig_mean-normal_tm1,trig_max-low_tm1,trig_max-high_tm1,trig_max-normal_tm1,gagne_sum_tm1,gagne_sum_t
0,0,1.987430,0,1200.0,0.0,NaN,5.4,NaN,1.110000,194.0,...,0,0,0,0,0,0,0,0,0,0
1,1,7.677934,0,2600.0,0.0,119.0,5.5,40.4,0.860000,93.0,...,0,1,0,0,1,0,0,1,4,3
2,2,0.407678,0,500.0,0.0,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,3,0.798369,0,1300.0,0.0,117.0,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,4,17.513165,0,1100.0,0.0,116.0,NaN,34.1,1.303333,53.0,...,0,0,0,0,0,0,0,0,1,1


In [ ]:
%cd /content/drive/MyDrive/Github/

[Errno 2] No such file or directory: '/content/drive/MyDrive/Github/'
/content
